In [1]:
import torch
import tqdm
import einops
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import transforms, datasets

In [2]:
class Residual(nn.Module):
    def __init__(self, fn):
        super().__init__()
        self.fn = fn

    def forward(self, x):
        return self.fn(x) + x

In [3]:
class LayerNormalize(nn.Module):
    def __init__(self, dim, fn):
        super().__init__()
        self.fn = fn
        self.norm = nn.LayerNorm(dim)

    def forward(self, x):
        return self.fn(self.norm(x))

In [4]:
class MLP(nn.Module):
    def __init__(self, dim, hidden_dim):
        super().__init__()
        self.gelu = nn.GELU()
        self.dropout = nn.Dropout(0.1)

        self.fc1 = nn.Linear(dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, dim)
        self.init_parameters()
        
    def init_parameters(self):
        nn.init.xavier_uniform_(self.fc1.weight)
        nn.init.normal_(self.fc1.bias, std=1e-6)
        
        nn.init.xavier_uniform_(self.fc2.weight)
        nn.init.normal_(self.fc2.bias, std=1e-6)


    def forward(self, x):
        x = self.gelu(self.fc1(x))
        x = self.dropout(x)

        x = self.fc2(x)
        x = self.dropout(x)

        return x

In [5]:
class MultiHeadAttention(nn.Module):
    def __init__(self, dim, heads):
        super().__init__()
        self.heads = heads
        self.scale = dim ** -0.5
        self.dropout = nn.Dropout(0.1)

        self.fc1 = nn.Linear(dim, dim*3)
        self.fc2 = nn.Linear(dim, dim)
        self.init_parameters()

    def init_parameters(self):
        nn.init.xavier_uniform_(self.fc1.weight)
        nn.init.zeros_(self.fc1.bias)
        
        nn.init.xavier_uniform_(self.fc2.weight)
        nn.init.zeros_(self.fc2.bias)

    def forward(self, x):
        qkv = self.fc1(x)
        q, k, v = einops.rearrange(qkv, "b n (qkv h d) -> qkv b h n d", qkv=3, h=self.heads)
        energy = torch.einsum("bhid,bhjd->bhij", q, k) * self.scale
        energy = energy.softmax(dim=-1)

        attn = torch.einsum("bhij,bhjd->bhid", energy, v)
        attn = einops.rearrange(attn, "b h n d -> b n (h d)")
        out = self.dropout(self.fc2(attn))

        return out

In [6]:
class Transformer(nn.Module):
    def __init__(self, depth, dim, hidden_dim, heads):
        super().__init__()
        self.layers = nn.ModuleList([])

        for _ in range(depth):
            self.layers.append(nn.ModuleList([Residual(LayerNormalize(dim, MultiHeadAttention(dim, heads))), 
                                              Residual(LayerNormalize(dim, MLP(dim, hidden_dim)))]))
            
    def forward(self, x):
        for attention, mlp in self.layers:
            x = attention(x)
            x = mlp(x)

        return x

In [7]:
class VisionTransformer(nn.Module):
    def __init__(self, in_channels, image_size, patch_size, num_classes, dim, depth, heads, hidden_dim):
        super().__init__()
        assert image_size % patch_size == 0, "Image Size must be divisible by Patch Size!!"

        num_patches = (image_size // patch_size) ** 2
        self.dropout = nn.Dropout(0.1)
        self.identity = nn.Identity()
        # self.gelu = nn.GELU()

        self.patch_embedding = nn.Conv2d(in_channels, dim, patch_size, stride=patch_size)
        self.cls_tokens = nn.Parameter(torch.zeros(1, 1, dim))
        self.pos_embedding = nn.Parameter(torch.empty(1, (num_patches + 1), dim))
        
        self.transformer = Transformer(depth, dim, hidden_dim, heads)
        self.fc = nn.Linear(dim, num_classes) # self.fc = nn.Linear(dim, hidden_dim) -> self.fc_out

        # self.fc_out = nn.Linear(hidden_dim, num_classes) -> use only for large datasets
        self.init_parameters()

    def init_parameters(self):
        nn.init.normal_(self.pos_embedding, std=0.02)

        nn.init.xavier_uniform_(self.fc.weight)
        nn.init.normal_(self.fc.bias, std=1e-6)

        # nn.init.xavier_uniform_(self.fc_out.weight)
        # nn.init.normal_(self.fc_out.bias)

    def forward(self, x):
        x = self.patch_embedding(x)
        x = einops.rearrange(x, "b c h w -> b (h w) c")

        cls_tokens = self.cls_tokens.expand(x.shape[0], -1, -1)
        x = torch.cat((cls_tokens, x), dim=1)
        x += self.pos_embedding
        x = self.dropout(x)

        x = self.transformer(x)
        x = self.identity(x[:, 0])

        x = self.fc(x)
        # x = self.dropout(self.gelu(x))
        # x = self.dropout(self.fc_out(x))

        return x

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
epochs = 30
batch_size = 100
in_channels = 3
image_size = 32
patch_size = 4
num_classes = 10
dim = 64
depth = 6
heads = 8
hidden_dim = 128
lr = 3e-3

In [9]:
device

device(type='cuda')

In [10]:
transform = transforms.Compose([transforms.ToTensor(), 
                                transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])

In [11]:
train_data = datasets.CIFAR10("data/", train=True, download=True, transform=transform)
test_data = datasets.CIFAR10("data/", train=False, download=True, transform=transform)

Files already downloaded and verified
Files already downloaded and verified


In [12]:
train_batches = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_batches = DataLoader(test_data, batch_size=batch_size, shuffle=False)

In [13]:
len(train_batches), len(test_batches)

(500, 100)

In [14]:
net = VisionTransformer(in_channels, image_size, patch_size, num_classes, dim, depth, heads, hidden_dim).to(device)
net

VisionTransformer(
  (dropout): Dropout(p=0.1, inplace=False)
  (identity): Identity()
  (patch_embedding): Conv2d(3, 64, kernel_size=(4, 4), stride=(4, 4))
  (transformer): Transformer(
    (layers): ModuleList(
      (0): ModuleList(
        (0): Residual(
          (fn): LayerNormalize(
            (fn): MultiHeadAttention(
              (dropout): Dropout(p=0.1, inplace=False)
              (fc1): Linear(in_features=64, out_features=192, bias=True)
              (fc2): Linear(in_features=64, out_features=64, bias=True)
            )
            (norm): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
          )
        )
        (1): Residual(
          (fn): LayerNormalize(
            (fn): MLP(
              (gelu): GELU()
              (dropout): Dropout(p=0.1, inplace=False)
              (fc1): Linear(in_features=64, out_features=128, bias=True)
              (fc2): Linear(in_features=128, out_features=64, bias=True)
            )
            (norm): LayerNorm((64,), eps

In [15]:
opt = torch.optim.Adam(net.parameters(), lr)
loss_fn = nn.CrossEntropyLoss(reduction="sum")

In [16]:
input = torch.randn(100, 3, 32, 32).to(device)
output = net(input)
output.shape

torch.Size([100, 10])

In [17]:
def get_accuracy(preds, y):
    preds = preds.argmax(dim=1, keepdim=True)
    correct = preds.squeeze(1).eq(y)
    acc = correct.sum() / torch.FloatTensor([y.shape[0]]).to(device)

    return acc.item()

In [18]:
def loop(net, batches, train):
    batch_losses = []
    batch_accs = []

    if train:
        print("Train Loop:")
        print("")
        net.train()

        for X, y in tqdm.tqdm(batches, total=len(batches)):
            X = X.to(device)
            y = y.to(device)

            preds = net(X)
            loss = loss_fn(preds, y)
            acc = get_accuracy(preds, y)

            opt.zero_grad()
            loss.backward()
            opt.step()

            batch_losses.append(loss.item())
            batch_accs.append(acc)

    else:
        print("Validation Loop:")
        print("")
        net.eval()

        with torch.no_grad():
            for X, y in tqdm.tqdm(batches, total=len(batches)):
                X = X.to(device)
                y = y.to(device)

                preds = net(X)
                loss = loss_fn(preds, y)
                acc = get_accuracy(preds, y)

                batch_losses.append(loss.item())
                batch_accs.append(acc) 

    print("")
    print("")
    
    return sum(batch_losses) / len(batch_losses), sum(batch_accs) / len(batch_accs)

In [19]:
for epoch in range(epochs):
    train_loss, train_acc = loop(net, train_batches, True)
    val_loss, val_acc = loop(net, test_batches, False)
    
    print(f"epoch: {epoch} | train_loss: {train_loss:.4f} | train_acc: {train_acc:.4f} | val_loss: {val_loss:.4f} | val_acc: {val_acc:.4f}")
    print("")

  0%|          | 2/500 [00:00<00:39, 12.58it/s]

Train Loop:



  4%|▍         | 4/100 [00:00<00:02, 32.70it/s]



Validation Loop:



  0%|          | 2/500 [00:00<00:29, 16.67it/s]

...



epoch: 28 | train_loss: 56.6283 | train_acc: 0.7977 | val_loss: 89.6068 | val_acc: 0.7222

Train Loop:



  4%|▍         | 4/100 [00:00<00:02, 34.42it/s]



Validation Loop:



100%|██████████| 100/100 [00:03<00:00, 32.18it/s]



epoch: 29 | train_loss: 55.4899 | train_acc: 0.7998 | val_loss: 89.6653 | val_acc: 0.7192

